In [1]:
import chess
import chess.engine

import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

https://python-chess.readthedocs.io/en/latest/

https://python-chess.readthedocs.io/en/latest/core.html

In [2]:
STOCKFISH = chess.engine.SimpleEngine.popen_uci("Stockfish/stockfish_13_win_x64_bmi2")

def stockfish_evaluation(board, depth=25):
    info = STOCKFISH.analyse(board, chess.engine.Limit(depth=depth))
    board_eval = info['score'].white().score(mate_score=100000)
    return board_eval


In [3]:
class ChessDataset(Dataset):
    """Chess Positions Evaluation Dataset"""

    def __init__(self, csv_file):
        self.df = pd.read_csv(csv_file)
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        position = np.array(self.df.iloc[idx, :-1])
        eval_ = np.array(self.df.iloc[idx, -1])
        
        sample = {'position': torch.from_numpy(position), 'eval': torch.from_numpy(eval_)}

        return sample

In [4]:
dataset = ChessDataset(csv_file='data/stockfish_depth0b.csv')

In [5]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__() 
        self.conv1 = nn.Conv2d(8, 8, 2) #input, output channels, kernel_size
        self.conv2 = nn.Conv2d(8, 8, 3)
        self.conv3 = nn.Conv2d(8, 8, 3)
        self.conv4 = nn.Conv2d(8, 8, 3)
        self.fc1 = nn.Linear(8, 64) 
        self.fc2 = nn.Linear(64, 1) 

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)   
        x = torch.reshape(x, (-1,)) # flattens into 1d tensor
        x = self.fc1(F.relu(x))
        x = self.fc2(torch.sigmoid(x))

        return x

In [6]:
net = Net()

# TRAINING DATA

In [7]:
optimizer = optim.Adam(net.parameters(), lr = 0.0009) #lr = 0.001
criterion = nn.MSELoss() 

In [8]:
trainloader = DataLoader(dataset, batch_size=1,
                        shuffle=True, num_workers=0)

for epoch in range(3):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs = data['position'].reshape(1, 8, 8, 8).float()
        labels = data['eval'].float()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 116140.589
[1,   400] loss: 98904.811
[1,   600] loss: 91828.034
[1,   800] loss: 85079.989
[1,  1000] loss: 84613.862
[1,  1200] loss: 94128.846
[1,  1400] loss: 97845.507
[1,  1600] loss: 97911.485
[1,  1800] loss: 86998.213
[2,   200] loss: 90453.227
[2,   400] loss: 105959.197
[2,   600] loss: 89171.011
[2,   800] loss: 101076.283
[2,  1000] loss: 97175.078
[2,  1200] loss: 95202.558
[2,  1400] loss: 90200.010
[2,  1600] loss: 86068.592
[2,  1800] loss: 92916.060
[3,   200] loss: 86885.025
[3,   400] loss: 109167.418
[3,   600] loss: 86882.109
[3,   800] loss: 88128.111
[3,  1000] loss: 96024.440
[3,  1200] loss: 115308.696
[3,  1400] loss: 86357.849
[3,  1600] loss: 92519.574
[3,  1800] loss: 98740.458
Finished Training
